<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>

# AI Agents & Automation — Chapter 8
## Multi-Agent Collaboration

&copy; Dr. Yves J. Hilpisch<br>
AI-Powered by GPT-5.

### Overview

This notebook accompanies Chapter 8 — Multi Agent Collab. It is self-contained and demonstrates the core ideas with small, readable code cells. Run cells from top to bottom; each code cell is preceded by a short explanation of what it does.


Two tiny agents — a planner and a critic — exchange short messages via a minimal bus.

In [ ]:
from dataclasses import dataclass  # import dataclass helper
from typing import List  # import typing helpers

@dataclass  # structured chat envelope
class Message:
    role: str  # sender role
    content: str  # message body
    tags: List[str]  # annotations

class Bus:  # tiny in-memory message bus
    def __init__(self) -> None:  # initialize transcript
        self.log: List[Message] = []  # store dialogue

    def send(self, message: Message) -> None:  # append + log message
        self.log.append(message)
        # echo for logging
        print({
            'role': message.role,
            'tags': message.tags,
            'content': message.content,
        })

def planner(previous: Message | None) -> Message:  # propose or revise plan
    if previous is None:
        # first move
        return Message(
            'planner',
            'Plan: parse input -> draft report',
            ['proposal'],
        )
    if 'revise' in previous.tags:
        # revised plan
        return Message(
            'planner',
            'Plan: parse input -> summarize -> draft report',
            ['proposal'],
        )
    return Message('planner', 'Ready to hand off execution', ['handoff'])  # default

def critic(previous: Message) -> Message:  # inspect planner output
    if 'report' in previous.content:
        return Message('critic', 'Looks good', ['approve'])  # approve
    # request revision
    return Message(
        'critic',
        'Please include report step',
        ['revise'],
    )

bus = Bus()  # create message bus
last: Message | None = None  # track last message
for turn in range(4):  # bounded dialogue budget
    message = planner(last) if turn % 2 == 0 else critic(last)  # alternate roles
    bus.send(message)  # record message
    if 'approve' in message.tags:
        break  # stop when approved
    last = message  # keep latest message


<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>